# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "../WeatherPy/output_data/cities.csv"
with open(output_data_file, "r", encoding="utf-8") as read_file:
    data_df = pd.read_csv(read_file)

data_df.head()
 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Camacha,33.08,-16.33,68.00,88,75,13.87,PT
1,Mataura,-46.19,168.86,42.75,93,100,6.22,NZ
2,Hobart,-42.88,147.33,44.60,81,20,5.82,AU
3,Manacor,39.57,3.21,73.40,94,0,3.36,ES
4,São José da Coroa Grande,-8.90,-35.15,75.60,81,2,6.60,BR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = data_df[["Lat", "Lng"]]
humidity = data_df["Humidity"]

In [4]:
# Create heat layer
fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#setting my ideal weather conditions 
vacation_city = data_df.loc[(data_df["Max Temp"] <= 80)&(data_df["Max Temp"]>=74)&
                            (data_df["Humidity"] <= 70)&(data_df["Cloudiness"]<=20)&
                            (data_df["Wind Speed"] <10)] 
                                                             
vacation_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
68,Mingora,34.78,72.36,75.87,38,0,5.97,PK
116,Hibbing,47.43,-92.94,78.80,61,20,4.70,US
170,Bismil,37.85,40.67,75.20,56,20,3.36,TR
179,Jamestown,42.10,-79.24,79.00,54,1,8.05,US
208,Normandin,48.83,-72.53,75.20,60,20,1.32,CA
312,Chapais,49.78,-74.85,77.00,53,1,8.05,CA
437,Kapoeta,4.77,33.59,76.71,51,18,5.59,SS
442,Mayor Pablo Lagerenza,-19.93,-60.77,77.23,56,4,8.88,PY
532,Hyères,43.12,6.12,79.00,57,0,4.70,FR
538,Wana,32.30,69.57,77.50,53,0,1.86,PK


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_city 
hotel_names = []
hotel_df["Hotel Name"] = np.nan
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type": "hotel","key": g_key,"radius": 5000}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    loc = f"{row['Lat']},{row['Lng']}"
    params["location"] = loc
    response = requests.get(base_url, params=params).json()    
    hotel_names.append(response['results'][1]['name'])
    
hotel_df["Hotel Name"] = hotel_names
hotel_df






C:\Users\whl42\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\whl42\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
68,Mingora,34.78,72.36,75.87,38,0,5.97,PK,Butkara I
116,Hibbing,47.43,-92.94,78.80,61,20,4.70,US,WMFG-FM Hibbing
170,Bismil,37.85,40.67,75.20,56,20,3.36,TR,Tekin Halı - Perde - Ev Tekstili
179,Jamestown,42.10,-79.24,79.00,54,1,8.05,US,Hampton Inn & Suites Jamestown
208,Normandin,48.83,-72.53,75.20,60,20,1.32,CA,Banque Nationale
312,Chapais,49.78,-74.85,77.00,53,1,8.05,CA,Hôtel Opémiska
437,Kapoeta,4.77,33.59,76.71,51,18,5.59,SS,Kapoeta Police Station
442,Mayor Pablo Lagerenza,-19.93,-60.77,77.23,56,4,8.88,PY,Estación Meteorológica Automática
532,Hyères,43.12,6.12,79.00,57,0,4.70,FR,ibis budget Hyères Centre-Ville
538,Wana,32.30,69.57,77.50,53,0,1.86,PK,UAE PAP Cadet College Wana E2-2


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))